In [4]:
#Python program to scrape cricket Women's rankings from ‘www.icc-cricket.com’
# -*- coding: utf-8 -*-
!pip install bs4
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

headers = {
	"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
	}

urls = [
"https://www.icc-cricket.com/rankings/mens/team-rankings/odi",
"https://www.icc-cricket.com/rankings/womens/team-rankings/odi"
]


for url in urls:
    request_object = requests.get(url, headers=headers)
    html_content = request_object.text
    print(request_object.status_code, "->", url)
    soup_object = BeautifulSoup(html_content, "html.parser")
    for element in soup_object.select('[class="ranking-pos up"], [class="ranking-pos down"]'):
        element.replace_with(BeautifulSoup("<" + element.name + "></" + element.name + ">", "html.parser"))

    ranking_type = soup_object.select_one(".rankings-block__title-container > h4").text
 
    results_file_name = ranking_type + ".csv"
    columns_names = ["Position", "Team Name", "Matches","Points","Rating","Crawl URL"]
    pd.DataFrame(columns=columns_names).to_csv(results_file_name, sep=",", index=False, encoding="utf-8")
            
    for element in soup_object.select('div[class="rankings-block__container full rankings-table"] tr'):
        if(element.find("th")):
            continue
        data_dicts = dict()
        data_dicts["Crawl URL"] = url
        data_dicts["Ranking Type"] = ranking_type
        if(element.select_one('[class*="position"],[class*="pos"]')):
            data_dicts["Position"] = element.select_one('[class*="position"],[class*="pos"]').text
        if(element.select_one('[class^="flag-15"],[class^="flag-30"]')):
            data_dicts["Team Name"] = element.select_one('[class^="flag-15"],[class^="flag-30"]')["class"][-1]
        if(element.select_one('[class*="matches"],[class*="table-body__cell u-center-text"]')):
            data_dicts["Matches"] = element.select_one('[class*="matches"],[class*="table-body__cell u-center-text"]').text            
        for pointss in (element.select('td:nth-of-type(4)')):
            if(pointss.text.strip()):
                data_dicts["Points"] = pointss.text        
        if(element.select_one('[class$="rating"],[class$="rankings-block__banner--rating u-text-right"]')):
            data_dicts["Rating"] = element.select_one('[class$="rating"],[class$="rankings-block__banner--rating u-text-right"]').text
        for key in data_dicts.keys():
            data_dicts[key] = re.sub(r"\s+", " ", data_dicts[key])
            data_dicts[key] = data_dicts[key].strip()
        pd.DataFrame([data_dicts], columns=columns_names).to_csv(results_file_name, sep=",", index=False, header=False, encoding="utf-8", mode="a")


200 -> https://www.icc-cricket.com/rankings/mens/team-rankings/odi
200 -> https://www.icc-cricket.com/rankings/womens/team-rankings/odi
